In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [4]:
training_data = []
lines=[]
with open('final_training_data.json', 'r') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
      for annotation in data['annotation']:
          if(annotation is not None):
            #only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [5]:
len(training_data)

446

In [11]:
training_data[0:5]

('PCP is YUVRAJ KUMAR, MC,', {'entities': [(7, 19, 'PERSON')]})

In [15]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 446/446 [00:00<00:00, 228826.86it/s]


Starting iteration 0


100%|██████████| 446/446 [00:05<00:00, 82.24it/s]


{'ner': 909.0319382282688}
Starting iteration 1


100%|██████████| 446/446 [00:05<00:00, 83.78it/s]


...

{'ner': 114.56468431414288}
Starting iteration 18


100%|██████████| 446/446 [00:05<00:00, 82.28it/s]


{'ner': 115.77984307230383}
Starting iteration 19


100%|██████████| 446/446 [00:05<00:00, 83.60it/s]

{'ner': 94.54388660158337}


In [33]:
## Training pre trained spacy model.

nlp2 = spacy.load('en_core_web_sm')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp2.pipe_names:
    ner = nlp2.create_pipe('ner')
    nlp2.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp2.pipe_names if pipe != 'ner']
with nlp2.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp2.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp2.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 446/446 [00:00<00:00, 678758.92it/s]


Starting iteration 0


100%|██████████| 446/446 [00:06<00:00, 66.62it/s]


{'ner': 381.9081055177883}
Starting iteration 1


100%|██████████| 446/446 [00:06<00:00, 68.86it/s]


...

{'ner': 71.26947862512203}
Starting iteration 18


100%|██████████| 446/446 [00:06<00:00, 68.77it/s]


{'ner': 82.94847494132499}
Starting iteration 19


100%|██████████| 446/446 [00:06<00:00, 68.15it/s]

{'ner': 68.2165328378857}


In [34]:
# Saving the trained model to disk.
nlp2.to_disk('trained_spacy_model2')

This PRON nsubj
is AUX ROOT
a DET det
test NOUN compound
sentence NOUN attr
. PUNCT punct


ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.